In [1]:
import pandas as pd
import numpy as np
import pyarrow.parquet as parquet

In [2]:
# Where the downloaded data are
input_path = '/home/adeykin/projects/SNAHackaton'
# Where to store results
output_path = '/home/adeykin/projects/SNAHackaton/out'

In [3]:
test = parquet.read_table(input_path + '/imagesTest', \
                          columns = ['instanceId_userId', 'instanceId_objectId','ImageId']).to_pandas()
test.head(10)

,instanceId_userId,instanceId_objectId,ImageId
0,602,25708174,[d215ecd5da0a088db89b97db3b6e811a]
1,602,24393746,[79eacdd5e803ad80d1c62de4047d74ad]
2,803,25400331,[771591947cf0a72be0702fe53e83fe11]
3,803,36083673,[86a85b7f9a4f528e87dececd0b400fac]
4,803,33713302,[244ed11c75d01ab4c22ece5b6ab72f39]
5,3065,22946340,[269d9e263e34733e92944380a12b23ac]
6,3065,38325772,[056a349e7c71694baab197b79fe9b7d3]
7,3065,36426827,[24fec4db493603e1d546bcb6b71a5ae3]
8,3065,37549050,[9bc47732b71a66f12248649eef291ed5]
9,3065,20719945,[8e16231d4adb8e91d4fc13d949f1e2ce]


In [4]:
test['ImageId'] = test['ImageId'].map(lambda x: x[0])

In [5]:
prediction = pd.read_pickle('predictLinear.pkl')

In [6]:
prediction.shape

(352505, 2)

In [7]:
df = pd.merge(left=test, right=prediction, left_on='ImageId', right_on='fileName')

In [62]:
test[test['instanceId_userId']==81068]

,instanceId_userId,instanceId_objectId,ImageId
1052816,81068,39130491,ffc2840a51af8d8775952b6dc0596297
1052817,81068,30683059,44f4c7ffb79945263d29e584bfc135b3


In [64]:
prediction[prediction['fileName'] == '44f4c7ffb79945263d29e584bfc135b3']

,fileName,score
95326,44f4c7ffb79945263d29e584bfc135b3,1.381083


In [65]:
prediction[prediction['fileName'] == 'ffc2840a51af8d8775952b6dc0596297']

,fileName,score
352351,ffc2840a51af8d8775952b6dc0596297,1.174339


In [66]:
df[df['instanceId_userId']==81068]

,instanceId_userId,instanceId_objectId,ImageId,fileName,score
83173,81068,30683059,44f4c7ffb79945263d29e584bfc135b3,44f4c7ffb79945263d29e584bfc135b3,1.381083
98410,81068,39130491,ffc2840a51af8d8775952b6dc0596297,ffc2840a51af8d8775952b6dc0596297,1.174339


In [67]:
test['ImageId'].unique().shape

(209109,)

In [68]:
prediction['fileName'].unique().shape

(352505,)

In [38]:
index1 = pd.Index(test['ImageId'].unique())

In [29]:
index2 = pd.Index(prediction['fileName'])

In [41]:
d = index1.difference(index2)
d

Index(['ffa1e48d38ba1a9b4e467827824c6aa3', 'ffa1efa525510797fdd79d3948516533',
       'ffa285cfd9daf34557a923b956751f07', 'ffa29a3cde90d43d51c7e983b21c5846',
       'ffa2bf4f6386a13b4242feea935a1294', 'ffa2fa0f58a826ecfd67ddba2ee356b1',
       'ffa3169c04d1156cf631592ceed3d654', 'ffa345c11a0db89196824bd32df2c7f9',
       'ffa3d3670eb6630e1beaf9bdea772192', 'ffa3d6ff3ba68f249605cfc0aaeca2cf',
       ...
       'fffd91bb314529124b460c6b7f67d069', 'fffdeda85d1ab35f339e48965624633e',
       'fffe70bbd6fb598d14867960c951a6f0', 'fffe86950f93056dfeb34071e72a1473',
       'ffff1b3e86ad950f1d2859f12ddea86d', 'ffff2678f0920341dc88155a7023b5d5',
       'ffff2fff03647b647b7de95c31e2db1c', 'ffff751415b974c0a8f1e771d8d91a1e',
       'ffffb57a60944c68b2bf15bce4bbf931', 'ffffb90d6cea12b3cc2d637738635abd'],
      dtype='object', length=295)

In [44]:
d[d == 'ffc2840a51af8d8775952b6dc0596297']

Index(['ffc2840a51af8d8775952b6dc0596297'], dtype='object')

In [51]:
d.

Index(['ffa1e48d38ba1a9b4e467827824c6aa3', 'ffa1efa525510797fdd79d3948516533',
       'ffa285cfd9daf34557a923b956751f07', 'ffa29a3cde90d43d51c7e983b21c5846',
       'ffa2bf4f6386a13b4242feea935a1294', 'ffa2fa0f58a826ecfd67ddba2ee356b1',
       'ffa3169c04d1156cf631592ceed3d654', 'ffa345c11a0db89196824bd32df2c7f9',
       'ffa3d3670eb6630e1beaf9bdea772192', 'ffa3d6ff3ba68f249605cfc0aaeca2cf',
       ...
       'fffd91bb314529124b460c6b7f67d069', 'fffdeda85d1ab35f339e48965624633e',
       'fffe70bbd6fb598d14867960c951a6f0', 'fffe86950f93056dfeb34071e72a1473',
       'ffff1b3e86ad950f1d2859f12ddea86d', 'ffff2678f0920341dc88155a7023b5d5',
       'ffff2fff03647b647b7de95c31e2db1c', 'ffff751415b974c0a8f1e771d8d91a1e',
       'ffffb57a60944c68b2bf15bce4bbf931', 'ffffb90d6cea12b3cc2d637738635abd'],
      dtype='object', length=295)

In [49]:
f = open('overextractList.txt', 'w')
for el in d:
    f.write(el + '\n')

In [39]:
index1

Index(['d215ecd5da0a088db89b97db3b6e811a', '79eacdd5e803ad80d1c62de4047d74ad',
       '771591947cf0a72be0702fe53e83fe11', '86a85b7f9a4f528e87dececd0b400fac',
       '244ed11c75d01ab4c22ece5b6ab72f39', '269d9e263e34733e92944380a12b23ac',
       '056a349e7c71694baab197b79fe9b7d3', '24fec4db493603e1d546bcb6b71a5ae3',
       '9bc47732b71a66f12248649eef291ed5', '8e16231d4adb8e91d4fc13d949f1e2ce',
       ...
       '9df60c38df7b44321409c04da3475dee', '36ee7bdadfc1d66518a1fdf100413351',
       '1f9a9b9d1e39b0a5ff114527e02c1597', '1ce872d2291a52f8f1970fa1566194f6',
       'ebf2820249d4ec44d9c216faff579ca4', '8d9d769fb79d9b582cd203991d87f122',
       'f4493c90da371e1afcf02d45aa48c927', 'b9e72e35ee742e608b55ee5010aeadc2',
       '7f9a5ee06efea72d69254884c246262f', '1e1056d1d3fbf3b8f6cf81c4726cfc09'],
      dtype='object', length=209109)

In [40]:
index2

Index(['0000247ff80ff7ba0eda80b611fec196', '000042fffe32f5b13211e9e6e44e5402',
       '0000d2dacea1db87d58336b5a3411724', '0000e46b41cdfa07fc87bf34434df10c',
       '00010a1fb0f14cf2e2acd1a94bfa3a58', '0001342fab36bfb25799e7a06f313b99',
       '00015e88eaab7b82715634aa0d94e4c3', '0001b4a10d1a9e95fe892709dd5b1f77',
       '0001bc02f2c701aaed3ad93869007aed', '0001d6ef22e79efa27a5ee8b4e1f96fe',
       ...
       'ff9fca0e3bfa05f87bba3c32cf67f1a5', 'ff9fdf1c960d48fecdfcd891a091dfc3',
       'ff9ff62f05d9f40d4973735cc9a05991', 'ffa062e4175416aeb5f6d6b0abc0abb2',
       'ffa0a222c5fbe33eb5de0583d472afdd', 'ffa0d3daf2003a6806bde12cef3dba16',
       'ffa1314f898ffbcd62ef986fd7cc3adf', 'ffa1bab4fa8aee0af7d41935a241e3c1',
       'ffa1bd47a76277301afa3e1675255c5a', 'ffa1d065accce9d6a24e15291b791ac5'],
      dtype='object', name='fileName', length=352256)

In [8]:
df = df[['instanceId_userId', 'instanceId_objectId', 'score']]

In [70]:
df['score'] = -df['score']

In [9]:
df.head()

,instanceId_userId,instanceId_objectId,score
0,602,25708174,1.301579
1,14437766,25708174,1.301579
2,5271680,25708174,1.301579
3,5855396,25708174,1.301579
4,7150973,25708174,1.301579


In [10]:
df = df.sort_values(by=['instanceId_userId', 'score', 'instanceId_objectId'], ascending=True)

In [11]:
df.head()

,instanceId_userId,instanceId_objectId,score
126756,59,29096263,1.381083
21422,59,39141251,1.516859
296205,107,39090161,1.117538
434711,107,22664858,1.171649
1175021,107,35377184,1.231837


In [12]:
df[df['instanceId_userId']==81068]

,instanceId_userId,instanceId_objectId,score
98410,81068,39130491,1.174339
83173,81068,30683059,1.381083


In [13]:
submit = df.groupby("instanceId_userId")['instanceId_objectId'].apply(list)
submit.head(10)

instanceId_userId
59                                  [29096263, 39141251]
107    [39090161, 22664858, 35377184, 10501436, 20368...
158                                 [30223730, 33303760]
188                                 [30483180, 37981047]
254                                 [15200850, 24219820]
377    [16916552, 28882686, 38100230, 37077770, 36170...
527             [21421114, 39110394, 39098017, 38334144]
602                                 [25708174, 24393746]
803                       [33713302, 25400331, 36083673]
908                                 [21144274, 39092075]
Name: instanceId_objectId, dtype: object

In [14]:
submit.to_csv(output_path + "/linearModelSubmitReverse.csv.gz", header = False, compression='gzip')